In [5]:
#######################################
########## IMPORTS ####################
from localLibrary_AWSConnector import *
import pandas as pd
import numpy as np

In [2]:
#HELPER FUNCTIONS
def get_data_s3():
    data_list = []
    data_dict = {}

    # Iterate through all buckets
    for bucket in s3.buckets.all():
        # Iterate through all items
        for obj in s3.Bucket(bucket.name).objects.all():
            if('data/' in obj.key and obj.key != 'data/'):
                data_list.append(obj.key)
                
                # Save STM files as DataFrames
                #if('STM' in obj.key and 'non' not in obj.key): 
                objct = s3.Bucket(bucket.name).Object(obj.key).get()
                data_dict[obj.key] = pd.read_csv(objct['Body'], index_col=0)

    print(data_list)
    
    return data_dict

def convert_time_to_int(time):
    #print(time)
    if time != time:
        return None
    else:
        return int(''.join(c for c in time if c.isdigit()))

In [3]:
all_data = get_data_s3()

['data/CRM_STM.csv', 'data/CRM_lost.csv', 'data/CRM_nonSTM.csv', 'data/FTS_STM.csv', 'data/FTS_lost.csv', 'data/FTS_nonSTM.csv', 'data/MK_STM.csv', 'data/MK_lost.csv', 'data/MK_nonSTM.csv', 'data/SG_STM.csv', 'data/SG_lost.csv', 'data/SG_nonSTM.csv', 'data/YZ_STM.csv', 'data/YZ_lost.csv', 'data/YZ_nonSTM.csv']


In [ ]:
all_data.keys()

In [ ]:
for key in all_data.keys():
    print(key)
    print(all_data[key].describe(include='all'))
    print("=====================================================")

In [17]:
#MERGE all DF for STM
STM = None
nonSTM = None
lost = None


for key in all_data.keys():
    df = all_data[key]
    how = 'outer'
    if 'SG' in key:
        pivoted = pd.pivot_table(all_data[key], values=['TotalSeatGeekTransactions', 'TotalTicketVolume', 'TotalScannedTicketVolume', 'TotalTicketDollarValue'], index=['SSB_CRMSYSTEM_CONTACT_ID'],
                    columns=['cjsgActivityType', 'cjsgSecondaryTicketType'], aggfunc=np.sum)
        df = pd.DataFrame(pivoted.to_records())
        sg = pd.DataFrame()
        sg['SSB_CRMSYSTEM_CONTACT_ID'] = df['SSB_CRMSYSTEM_CONTACT_ID']
        sg['total_scanned'] = df["('TotalScannedTicketVolume', 'Purchase', 'Primary')"] + df["('TotalScannedTicketVolume', 'Purchase', 'Resale')"] + df["('TotalScannedTicketVolume', 'Purchase', 'Transfer')"]

        sg['primary_purchase_transactions'] = df["('TotalSeatGeekTransactions', 'Purchase', 'Primary')"]
        sg['secondary_purchase_transactions'] = df["('TotalSeatGeekTransactions', 'Purchase', 'Resale')"] + df["('TotalSeatGeekTransactions', 'Purchase', 'Transfer')"]
        sg['secondary_sell_transactions'] = df["('TotalSeatGeekTransactions', 'Sell', 'Resale')"] + df["('TotalSeatGeekTransactions', 'Sell', 'Transfer')"]

        sg['primary_purchase_dollars'] = df["('TotalTicketDollarValue', 'Purchase', 'Primary')"]
        sg['secondary_purchase_dollars'] = df["('TotalTicketDollarValue', 'Purchase', 'Resale')"] + df["('TotalTicketDollarValue', 'Purchase', 'Transfer')"]
        sg['secondary_sell_dollars'] = df["('TotalTicketDollarValue', 'Sell', 'Resale')"] + df["('TotalTicketDollarValue', 'Sell', 'Transfer')"]

        sg['primary_purchase_tickets'] = df["('TotalTicketVolume', 'Purchase', 'Primary')"]
        sg['secondary_purchase_tickets'] = df["('TotalTicketVolume', 'Purchase', 'Resale')"] + df["('TotalTicketVolume', 'Purchase', 'Transfer')"]
        sg['secondary_sell_tickets'] = df["('TotalTicketVolume', 'Sell', 'Resale')"] + df["('TotalTicketVolume', 'Sell', 'Transfer')"]
        
        df = sg
        how = 'right'
        
    if('STM' in key and 'non' not in key): 
        if STM is None:
            STM = df
        else:
            STM = STM.merge(df, how = how, on = 'SSB_CRMSYSTEM_CONTACT_ID')
    elif('non' in key): 
        if nonSTM is None:
            nonSTM = df
        else:
            nonSTM = nonSTM.merge(df, how = how, on = 'SSB_CRMSYSTEM_CONTACT_ID')
    elif('lost' in key): 
        if lost is None:
            lost = df
        else:
            lost = lost.merge(df, how = how, on = 'SSB_CRMSYSTEM_CONTACT_ID')
    else:
        pass

In [18]:
stm_null = pd.DataFrame(STM.isna().sum()/len(STM), columns = ['STM_Pct_Null'])
nonstm_null = pd.DataFrame(nonSTM.isna().sum()/len(nonSTM), columns = ['nonSTM_Pct_Null'])
lost_null = pd.DataFrame(lost.isna().sum()/len(lost), columns = ['lost_Pct_Null'])

all_null = stm_null.merge(nonstm_null.merge(lost_null, left_index=True, right_index=True), left_index=True, right_index=True)

In [19]:
all_null.sort_values(by = 'STM_Pct_Null', ascending = True)

,STM_Pct_Null,nonSTM_Pct_Null,lost_Pct_Null
SSB_CRMSYSTEM_CONTACT_ID,0.000000,0.000009,0.000000
primary_purchase_transactions,0.079293,0.648206,0.618321
primary_purchase_tickets,0.079293,0.648206,0.618321
primary_purchase_dollars,0.079293,0.648206,0.618321
TotalCRMActivityVolume,0.112121,0.233133,0.051236
EarliestCRM,0.112121,0.233133,0.051236
LatestCRM,0.112121,0.233133,0.051236
TotalMarketoVolume,0.133333,0.392229,0.167486
EarliestMarketoDate,0.133333,0.392229,0.167486
LatestMarketoDate,0.133333,0.392229,0.167486


In [20]:
print(f"STM Length: {len(STM)}")
print(f"nonSTM Length: {len(nonSTM)}")
print(f"lost Length: {len(lost)}")


STM Length: 1980
nonSTM Length: 106244
lost Length: 15458


In [ ]:
ax = STM.hist(bins=30, grid=False, figsize=(15,10), color='#86bf91', zorder=2, rwidth=0.9)

In [ ]:
STM['EarliestCRM_int'] = [convert_time_to_int(x) for x in STM['EarliestCRM']]
STM['LatestCRM_int'] = [convert_time_to_int(x) for x in STM['LatestCRM']]

STM['LatestSeatGeek_int'] = [convert_time_to_int(x) for x in STM['LatestSeatGeekDate']]
STM['EarliestSeatGeek_int'] = [convert_time_to_int(x) for x in STM['EarliestSeatGeekDate']]

STM['EarliestMarketo_int'] = [convert_time_to_int(x) for x in STM['EarliestMarketoDate']]
STM['LatestMarketo_int'] = [convert_time_to_int(x) for x in STM['LatestMarketoDate']]

STM['EarliestFanatics_int'] = [convert_time_to_int(x) for x in STM['EarliestFanaticsDate']]
STM['LatestFanatics_int'] = [convert_time_to_int(x) for x in STM['LatestFanaticsDate']]

STM['EarliestYinzcam_int'] = [convert_time_to_int(x) for x in STM['EarliestYinzcamDate']]
STM['LatestYinzcam_int'] = [convert_time_to_int(x) for x in STM['LatestYinzcamDate']]

In [ ]:
ax = STM.hist(bins=30, grid=False, figsize=(15,10), color='#86bf91', zorder=2, rwidth=0.9)

In [8]:
STM.head()

,SSB_CRMSYSTEM_CONTACT_ID,TotalCRMActivityVolume,EarliestCRM,LatestCRM,TotalFanaticsTransactions,TotalFanaticsProductQty,TotalFanaticsDollarValue,EarliestFanaticsDate,LatestFanaticsDate,TotalMarketoVolume,...,secondary_sell_transactions,primary_purchase_dollars,secondary_purchase_dollars,secondary_sell_dollars,primary_purchase_tickets,secondary_purchase_tickets,secondary_sell_tickets,TotalYinzcamVolume,EarliestYinzcamDate,LatestYinzcamDate
0,7237949E-8842-459E-BFAA-EA4FC4A56FCE,8.0,2013-11-13 20:43:36,2018-04-23 17:41:28,NaN,NaN,NaN,NaN,NaN,106.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,82EEAC21-D630-48B3-B5AB-A26D44225E04,1.0,2012-06-21 06:48:30,2012-06-21 06:48:30,NaN,NaN,NaN,NaN,NaN,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,74715C50-EF52-497B-8B9C-FF328DA164BC,20.0,2009-05-21 01:29:13,2018-02-06 20:35:58,NaN,NaN,NaN,NaN,NaN,99.0,...,NaN,0.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN
3,8D3B3783-C9E1-4251-9A54-5BE78B40FDA7,2.0,2009-05-21 01:29:13,2018-01-22 20:13:57,NaN,NaN,NaN,NaN,NaN,80.0,...,42.0,7158.0,NaN,6456.0,133.0,NaN,90.0,NaN,NaN,NaN
4,EEC568A0-F708-4AEF-A6B8-C8BF99772AA6,23.0,2009-05-21 01:29:13,2017-11-29 20:34:49,NaN,NaN,NaN,NaN,NaN,127.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
